# Phase 1 — SQL EDA (Hospital Readmission)

This notebook performs **exploratory data analysis (EDA)** and **SQL-based analysis** on the
UCI dataset *"Diabetes 130-US hospitals for years 1999–2008"*.

## Prerequisites
1. Create the DuckDB database by running (from the project root):
```bash
python src/data_ingest.py --download
```

2. Ensure the database exists at:
- `data/processed/readmission.duckdb`

Notes:
- Do **not** commit `data/` or `mlruns/` to git (they should be ignored).
- This dataset contains sensitive demographic attributes (race/age/gender). Use responsibly.


In [ ]:
from __future__ import annotations

from pathlib import Path
import duckdb
import pandas as pd
import matplotlib.pyplot as plt

def find_project_root(start: Path | None = None) -> Path:
    """Walk upwards to find the project root (folder containing 'data' and 'src')."""
    start = start or Path.cwd()
    cur = start.resolve()
    for _ in range(10):
        if (cur / "data").exists() and (cur / "src").exists():
            return cur
        if cur.parent == cur:
            break
        cur = cur.parent
    raise RuntimeError("Could not locate project root. Run the notebook from inside the project folder.")

ROOT = find_project_root()
DB_PATH = ROOT / "data" / "processed" / "readmission.duckdb"

print("Project root:", ROOT)
print("DB path:", DB_PATH)

if not DB_PATH.exists():
    raise FileNotFoundError(
        f"DuckDB database not found at {DB_PATH}.\n"
        "Run: python src/data_ingest.py --download"
    )

con = duckdb.connect(str(DB_PATH))


## 1) Basic sanity checks

In [ ]:
df_counts = con.execute('''
SELECT
  COUNT(*) AS n_encounters,
  COUNT(DISTINCT patient_nbr) AS n_patients,
  SUM(readmission_30d) AS n_readmit_30d
FROM encounters
''').df()

df_counts


In [ ]:
df_readmitted = con.execute('''
SELECT readmitted, COUNT(*) AS n
FROM encounters
GROUP BY readmitted
ORDER BY n DESC
''').df()

df_readmitted


In [ ]:
rate_30d = con.execute('SELECT AVG(readmission_30d) AS rate FROM encounters').fetchone()[0]
print(f"Readmission <30d rate: {rate_30d:.4f} ({rate_30d*100:.2f}%)")

plt.figure()
plt.bar(["readmission_30d"], [rate_30d])
plt.ylabel("Rate")
plt.title("Overall readmission (<30 days) rate")
plt.show()


## 2) Missingness overview (selected columns)

In [ ]:
# In this dataset, missing values were standardized to NULL in the ingest script.
# We'll compute missingness for a selected set of high-signal columns.

cols = [
    "race", "gender", "age",
    "medical_specialty", "payer_code",
    "diag_1", "diag_2", "diag_3",
    "max_glu_serum", "A1Cresult"
]

# Keep only columns that exist in the table (defensive)
existing_cols = set(con.execute("DESCRIBE encounters").df()["column_name"].tolist())
cols = [c for c in cols if c in existing_cols]

n_total = con.execute("SELECT COUNT(*) FROM encounters").fetchone()[0]

rows = []
for c in cols:
    missing = con.execute(f"SELECT SUM(CASE WHEN {c} IS NULL THEN 1 ELSE 0 END) FROM encounters").fetchone()[0]
    rows.append({"column": c, "missing": int(missing), "missing_pct": float(missing) / n_total})

df_missing = pd.DataFrame(rows).sort_values("missing_pct", ascending=False)
df_missing


## 3) SQL analysis (8–10 queries)

In [ ]:
def q(sql: str) -> pd.DataFrame:
    """Run a SQL query and return a pandas DataFrame."""
    return con.execute(sql).df()


### Q1 — Readmission rate by age group (CTE)

In [ ]:
q('''
WITH agg AS (
  SELECT
    age,
    COUNT(*) AS n,
    AVG(readmission_30d) AS readmit_rate_30d
  FROM encounters
  GROUP BY age
)
SELECT *
FROM agg
ORDER BY age
''')


In [ ]:
df_age = q('''
SELECT age, AVG(readmission_30d) AS rate
FROM encounters
GROUP BY age
ORDER BY age
''')

plt.figure()
plt.plot(df_age["age"], df_age["rate"], marker="o")
plt.xticks(rotation=45, ha="right")
plt.ylabel("Readmission <30d rate")
plt.title("Readmission rate by age group")
plt.tight_layout()
plt.show()


### Q2 — Readmission rate by gender

In [ ]:
q('''
SELECT
  gender,
  COUNT(*) AS n,
  AVG(readmission_30d) AS readmit_rate_30d
FROM encounters
GROUP BY gender
ORDER BY n DESC
''')


### Q3 — Readmission rate by race

In [ ]:
q('''
SELECT
  race,
  COUNT(*) AS n,
  AVG(readmission_30d) AS readmit_rate_30d
FROM encounters
GROUP BY race
ORDER BY readmit_rate_30d DESC
''')


### Q4 — Readmission rate by number of medications (binned via CASE + CTE)

In [ ]:
q('''
WITH binned AS (
  SELECT
    CASE
      WHEN num_medications < 10 THEN '0-9'
      WHEN num_medications < 20 THEN '10-19'
      WHEN num_medications < 30 THEN '20-29'
      WHEN num_medications < 40 THEN '30-39'
      ELSE '40+'
    END AS meds_bin,
    readmission_30d
  FROM encounters
)
SELECT
  meds_bin,
  COUNT(*) AS n,
  AVG(readmission_30d) AS readmit_rate_30d
FROM binned
GROUP BY meds_bin
ORDER BY meds_bin
''')


### Q5 — Readmission rate by number of procedures

In [ ]:
q('''
SELECT
  num_procedures,
  COUNT(*) AS n,
  AVG(readmission_30d) AS readmit_rate_30d
FROM encounters
GROUP BY num_procedures
ORDER BY num_procedures
''')


### Q6 — Top primary diagnoses among <30d readmissions (JOIN)

In [ ]:
q('''
SELECT
  d.diag_code,
  COUNT(*) AS n_readmit_30d
FROM encounters e
JOIN diagnoses_long d
  ON e.encounter_id = d.encounter_id
WHERE e.readmission_30d = 1
  AND d.diag_position = 1
GROUP BY d.diag_code
ORDER BY n_readmit_30d DESC
LIMIT 15
''')


### Q7 — Diagnosis-level readmission rate with ranking (WINDOW function)

In [ ]:
q('''
WITH diag_stats AS (
  SELECT
    d.diag_code,
    COUNT(*) AS n,
    AVG(e.readmission_30d) AS readmit_rate_30d
  FROM encounters e
  JOIN diagnoses_long d
    ON e.encounter_id = d.encounter_id
  WHERE d.diag_position = 1
  GROUP BY d.diag_code
  HAVING COUNT(*) >= 200
)
SELECT
  diag_code,
  n,
  readmit_rate_30d,
  ROW_NUMBER() OVER (ORDER BY readmit_rate_30d DESC) AS rank_by_rate
FROM diag_stats
ORDER BY readmit_rate_30d DESC
LIMIT 20
''')


### Q8 — Readmission rate by length of stay (time_in_hospital)

In [ ]:
q('''
SELECT
  time_in_hospital,
  COUNT(*) AS n,
  AVG(readmission_30d) AS readmit_rate_30d
FROM encounters
GROUP BY time_in_hospital
ORDER BY time_in_hospital
''')


### Q9 — Patients with frequent encounters and their readmission risk (CTE)

In [ ]:
q('''
WITH patient_counts AS (
  SELECT
    patient_nbr,
    COUNT(*) AS n_encounters,
    AVG(readmission_30d) AS readmit_rate_30d
  FROM encounters
  GROUP BY patient_nbr
)
SELECT *
FROM patient_counts
WHERE n_encounters >= 3
ORDER BY n_encounters DESC, readmit_rate_30d DESC
LIMIT 20
''')


### Q10 — Within each age group: top diagnoses by readmission rate (JOIN + WINDOW)

In [ ]:
q('''
WITH diag_by_age AS (
  SELECT
    e.age,
    d.diag_code,
    COUNT(*) AS n,
    AVG(e.readmission_30d) AS readmit_rate_30d
  FROM encounters e
  JOIN diagnoses_long d
    ON e.encounter_id = d.encounter_id
  WHERE d.diag_position = 1
  GROUP BY e.age, d.diag_code
  HAVING COUNT(*) >= 150
),
ranked AS (
  SELECT
    *,
    DENSE_RANK() OVER (PARTITION BY age ORDER BY readmit_rate_30d DESC) AS rnk
  FROM diag_by_age
)
SELECT age, diag_code, n, readmit_rate_30d, rnk
FROM ranked
WHERE rnk <= 3
ORDER BY age, rnk
''')


## Close the connection

In [ ]:
con.close()